In [5]:
!pip install selenium

In [6]:
# 키워드 검색
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/'+word
    return url

In [7]:
# 첫 번째 게시물 클릭
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
#select_first(driver)

In [15]:
# 다음 게시물 열기
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
#move_next(driver)

In [8]:
# 인스타그램 로그인
def instagram_login(id,pw):
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(id)

    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(pw)
    input_pw.submit()
    time.sleep(3)
    
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()   #로그인정보 저장 '나중에 하기' 버튼 클릭
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click() #알림 설정 '나중에 하기' 버튼 클릭

In [124]:
def check_category(content):
    category = 0
    
    drawing = re.findall('드로잉|미술|화실|그림|캘리그라피', content)
    cooking = re.search('요리|음료|푸드|커피', content)
    baking = re.search('베이킹|디저트|케이크|마카롱|간식', content)
    exercise = re.search('운동|건강|필라테스|요가', content)
    craft = re.search('공예|라탄|가죽|인형|가방|자수|가방',content)
    pottery = re.search('도예|도자기|글라스|세라믹|유리|그릇', content)
    soap = re.search('비누|캔들|방향제|향수', content)
    etc = re.search('사진|촬영|피아노|음악|꽃|원예', content)
    
    
    if (drawing):
        category = 1
    elif (baking):
        category = 3  # 베이킹이랑 요리랑 겹쳐서 순서 바꿨음
    elif (cooking):
        category = 2
    elif (exercise):
        category = 4
    elif (craft):
        category = 5
    elif (pottery):
        category = 6
    elif (soap):
        category = 7
    elif (etc):
        category = 8
    else:
        category = 0
    
    return category

In [125]:
CATEGORY = {1:'드로잉', 2:'요리, 음료', 3:'베이킹, 디저트', 4: '운동', 5:'공예', 6: '도예', 7:'비누, 캔들', 8:'기타', 0:'없음'}

CATEGORY[7]

'비누, 캔들'

In [126]:
# 게시물 정보 가져오기
# 닉네임, 작성 일시, 위치 정보, 해시태그

import re
from bs4 import BeautifulSoup
import unicodedata

def get_data(driver, n):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK>span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문의 해시태그 가져오기
    #tags= re.findall(r'#[^\s,\\]+', content)

    
    # 작성 일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['title']
    
    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
        
    # 이름(사용자 이름) 가져오기
    name = driver.find_element_by_css_selector(".sqdOP.yWX7d._8A5w5.ZIAjV").text
    
    #print("get_data : n = ", n)
    # 사용자 사진 가져오기
    userImgUrl = soup.select('div.Jv7Aj.mArmR.pZp3x')[0].img['src']
    #print(userImgUrl)
    #userImgUrl =''
    
    # 사진 가져오기
    if (n >= 42): # 한번에 가져올 수 있는 사진 max 42개
        n = n%42
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(2)
        
    imgUrl = soup.select('div.KL4Bh')[n].img['src']
    #print(imgUrl)
    #imgUrl =''
    
    # 카테고리 설정
    try:
        check = check_category(content)
        category = CATEGORY[check]
        time.sleep(3)
    except Exception as e:
        print("여기서 ", e)
    
#     if (category !=0):
#         print(CATEGORY[category])
        
    # 수집한 정보 저장
    #data = [category, imgUrl, name, userImgUrl, date, place, tags]
    data = [category, imgUrl, name, userImgUrl, date, place]
    
    return data

#get_data(driver, 1)

In [167]:
# 2. 키워드 검색
key_word = '원데이클래스'
url = insta_searching(key_word)

# 3. 검색 페이지 접속
driver.get(url)
time.sleep(4)

# 4. 첫 번째 게시글 열기
select_first(driver)

In [84]:
# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# 사용자 사진 가져오기
where = soup.select('div.Jv7Aj.mArmR.pZp3x')[0]
print(where)
for i in where:
    userImgUrl = i.img['src']
    print(userImgUrl)

<div class="Jv7Aj mArmR pZp3x"><div aria-disabled="true" class="RR-M- mrq0Z" role="button" tabindex="-1"><canvas class="CfWVH" height="53" style="position: absolute; top: -5px; left: -5px; width: 42px; height: 42px;" width="53"></canvas><a class="_2dbep qNELH kIKUG" href="/biahn.soap/" style="width: 32px; height: 32px;" tabindex="0"><img alt="biahn.soap님의 프로필 사진" class="_6q-tv" crossorigin="anonymous" data-testid="user-avatar" draggable="false" src="https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/212629673_356187162583892_674493385143858772_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&amp;_nc_ohc=nPkdTSZ0js0AX_FS5u-&amp;edm=AP_V10EBAAAA&amp;ccb=7-4&amp;oh=da6409db8d1251fca9ca8fd8a017b5cf&amp;oe=611913D1&amp;_nc_sid=4f375e"/></a></div></div>
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/212629673_356187162583892_674493385143858772_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=nPkdTSZ0js0AX_FS5u-&edm=AP_V10EBAAAA&ccb=7-4&oh=da6409db8d1251fca9ca8

In [107]:
# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

li = []
# 사진 가져오기

for scroll in range(5):
    where = soup.select('div.KL4Bh')

    #print(where)
    for i in where:
        imgUrl = i.img['src']
        li.append(imgUrl)

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(2)
print(len(li))

215


In [121]:
# for i in results:
#     print(i[0])
    
print("\nsize : ", len(results))


size :  89


In [115]:
from PIL import Image
import requests

for i in range(target):
    imgUrl = results[i][1]
    username = results[i][2]
    print(i,": ",username,imgUrl)

    im = Image.open(urlopen(imgUrl))
    im

0 :  mingmiraclegreen https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/233564092_166647708789273_2773787315642028626_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=wCc4AIHhcUwAX-zfIWj&edm=AP_V10EBAAAA&ccb=7-4&oh=f67ef4db09045529def36bcc3f4c15e0&oe=611926B8&_nc_sid=4f375e
1 :  ballet_jini https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/233684886_2607636926208982_5287449811268705697_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=qBWW_IeN1hwAX_DqO_H&edm=APU89FABAAAA&ccb=7-4&oh=4534c221009bed77839125cb4bd151f4&oe=61174B26&_nc_sid=86f79a
2 :  i_cd_i https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/233679785_4202144759900090_8464782836325507763_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=FrqpQ3bT2tEAX9ECHTJ&edm=AP_V10EBAAAA&ccb=7-4&oh=9c8e78263c62d54f7fc67112fd302ce5&oe=61180230&_nc_sid=4f375e
3 :  on_the1f https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080

In [117]:
from urllib.request import urlopen

for i in range(target):
    imgUrl = results[i][1]
    print(i,": ",imgUrl)
    with urlopen(imgUrl) as f:
        with open('./img/' + results[i][1] + str(i) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

0 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/233564092_166647708789273_2773787315642028626_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=wCc4AIHhcUwAX-zfIWj&edm=AP_V10EBAAAA&ccb=7-4&oh=f67ef4db09045529def36bcc3f4c15e0&oe=611926B8&_nc_sid=4f375e
1 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/233684886_2607636926208982_5287449811268705697_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=qBWW_IeN1hwAX_DqO_H&edm=APU89FABAAAA&ccb=7-4&oh=4534c221009bed77839125cb4bd151f4&oe=61174B26&_nc_sid=86f79a
2 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/233679785_4202144759900090_8464782836325507763_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=FrqpQ3bT2tEAX9ECHTJ&edm=AP_V10EBAAAA&ccb=7-4&oh=9c8e78263c62d54f7fc67112fd302ce5&oe=61180230&_nc_sid=4f375e
3 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/234136918_929415177611017_349565021456132975

In [122]:
for i in range(target):
    userImgUrl = results[i][3]
    print(i,": ",userImgUrl)
    with urlopen(userImgUrl) as f:
        with open('./user/' + 'user_'+results[i][1] + str(i) + '.jpg', 'wb') as h:
            img = f.read()
            h.write(img)

0 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/24127150_146214572807336_1879551480562712576_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=yxhA26mMRQYAX8SMEyI&edm=AP_V10EBAAAA&ccb=7-4&oh=6d91fbb00d6aa529a0ae6f245674a261&oe=611889C5&_nc_sid=4f375e
1 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/191204484_499943227916304_7848707498602884047_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=ymGWqv8ONyAAX89Heez&edm=AP_V10EBAAAA&ccb=7-4&oh=3d35b1a026cd4ba218eaa97bfa16f738&oe=6117F6EF&_nc_sid=4f375e
2 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/157267203_127317015980071_3131037788886948735_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=XMprA9fptEcAX_4Alf7&edm=AP_V10EBAAAA&ccb=7-4&oh=00ef0f1a5c78ef1b4803f0eea4d4fd6b&oe=6118E73D&_nc_sid=4f375e
3 :  https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/132005602_1647893762068942_4854535290532724068_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=

In [127]:
# 여러 게시글 정보 수집하기

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬 브라우저 열기 및 로그인
driver = webdriver.Chrome('c:/users/mindo/like_lion/oneday/chromedriver.exe')
site = 'https://www.instagram.com/'
driver.get(site)
time.sleep(2)

email = 'mindori1019@gmail.com'
password = 'alswn4249!'
instagram_login(email, password)

# 2. 키워드 검색
key_word = '원데이클래스'
url = insta_searching(key_word)

# 3. 검색 페이지 접속
driver.get(url)
time.sleep(4)

# 4. 첫 번째 게시글 열기
select_first(driver)

# 5. 여러 게시물 수집하기
results =[]

target = 5  # 크롤링할 게시글 수
for i in range(target):
    try:
        data = get_data(driver, i)
        if (data[0] == 0):
            move_next(driver)
        else:
            results.append(data)
            move_next(driver)
        
        #print(data[3])
        
#     except Exception as e:
#         print("예외?? ",e)
#         time.sleep(5)
#         move_next(driver)
    finally:
        print(i," : ", data[0]) 


0  :  베이킹, 디저트
1  :  비누, 캔들
2  :  운동
3  :  운동
4  :  비누, 캔들


In [145]:
# 5. 여러 게시물 수집하기
results =[]

target = 200  # 크롤링할 게시글 수
for i in range(target):
    try:
        data = get_data(driver, i)
        if (data[0] == '없음'):
            move_next(driver)
        elif (data[-1] == ''):
            #print('장소가 없어여')
            move_next(driver)
        else:
            results.append(data)
            move_next(driver)
        
        #print(data[3])
        
    except Exception as e:
        print("예외?? ",e)
        time.sleep(5)
        move_next(driver)
    finally:
        print(i," : ", data[0]) 

0  :  기타
1  :  기타
2  :  없음
3  :  기타
4  :  공예
5  :  공예
6  :  기타
7  :  없음
8  :  기타
9  :  베이킹, 디저트
10  :  없음
11  :  도예
12  :  기타
13  :  베이킹, 디저트
14  :  기타
15  :  베이킹, 디저트
16  :  없음
17  :  드로잉
18  :  베이킹, 디저트
19  :  없음
20  :  베이킹, 디저트
21  :  없음
22  :  기타
23  :  기타
24  :  없음
25  :  베이킹, 디저트
26  :  없음
27  :  기타
28  :  없음
29  :  공예
30  :  비누, 캔들
31  :  드로잉
32  :  베이킹, 디저트
33  :  베이킹, 디저트
34  :  없음
35  :  공예
36  :  도예
37  :  없음
38  :  공예
39  :  없음
40  :  없음
41  :  베이킹, 디저트
42  :  기타
43  :  없음
44  :  공예
45  :  비누, 캔들
46  :  기타
47  :  비누, 캔들
48  :  없음
49  :  공예
50  :  요리, 음료
51  :  도예
52  :  베이킹, 디저트
53  :  기타
54  :  베이킹, 디저트
55  :  없음
56  :  없음
57  :  기타
58  :  기타
59  :  기타
60  :  기타
61  :  공예
62  :  기타
63  :  운동
64  :  베이킹, 디저트
65  :  없음
66  :  비누, 캔들
67  :  기타
68  :  공예
69  :  기타
70  :  없음
71  :  없음
72  :  베이킹, 디저트
73  :  비누, 캔들
74  :  도예
75  :  없음
76  :  없음
77  :  기타
78  :  없음
79  :  베이킹, 디저트
80  :  기타
81  :  기타
82  :  베이킹, 디저트
83  :  기타
84  :  기타
85  :  베이킹, 디저트
86  :  기타
87  :  없음
88  :  없

KeyboardInterrupt: 

In [144]:
for i in results:
    print(i)

['비누, 캔들', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/235452436_191038883010640_318123959386751631_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=iLEJVh0CTE4AX_3xYod&edm=AMKDjl4BAAAA&ccb=7-4&oh=cb8575a5db740a74a1768b3caf64cc64&oe=6119F3A9&_nc_sid=1fe099&ig_cache_key=MjYzNjc3Njc2Njg3Mzg4MjI2OQ%3D%3D.2-ccb7-4', 'ang_yul', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-19/s150x150/110328572_211943336809538_3858693936088303740_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_ohc=W8O2eOAWGYwAX-4J8Cu&edm=AP_V10EBAAAA&ccb=7-4&oh=5ce3b42e8dcd33da032d489c1f463978&oe=6118443D&_nc_sid=4f375e', '2021년 8월 10일', '모란역']
['비누, 캔들', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/233463313_218928906699320_2118969279481270306_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=4hPtjgZloZAAX_Wfp4i&tn=IS_E9HmOyUFgjAcX&edm=AP_V10EBAAAA&ccb=7-4&oh=366a4d37593bf0e2cec372bb52168336&oe=6118848C&_nc_sid=4f375e', 'ovu_____', 'https:/

In [132]:
# 엑셀에 데이터 저장
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns =  ['category','photo','name', 'userphoto', 'date', 'place']
results_df.to_excel('c:/users/mindo/like_lion/oneday/Oneday_category2.xlsx', index = False)

In [44]:

# html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
    
# 본문 내용 가져오기
try:
    content = soup.select('div.C4VMK>span')[0].text
    content = unicodedata.normalize('NFC', content)
except:
    content = ''
    
# 본문의 해시태그 가져오기
tags= re.findall(r'#[^\s,\\]+', content)
    
print(content)

물레원데이클래스 완성작 #미니화병 🌸돌아가는 물레위에서 붓결을 내주고 싶어 하셨는데, 결과물을 보니 아주 잘 내주셨어요👏🏻 ...#물레수업 #도자기공방 #인천도자기공방#온더테이블 #주말데이트 #도자기원데이클래스 #도예공방 #인천데이트 #고유포터리 #일상 #물레체험 #인천데이트 #원데이클래스 #커플원데이클래스 #도자기원데이 #커플데이트 #좋아요 #주말체험 #인천데이트코스 #이색데이트 #인천취미 #인천 #도예원데이클래스 #도예체험 #인천공방 #구월동공방 #구월동 #물레원데이클래스 #코로나데이트 #코로나


In [45]:
c = check_category(content)
print(c)
print(CATEGORY[c])

6
도예


In [156]:
import re

soap = re.findall('베이킹|쿠킹',content)

if (soap):
    print(soap)
else:
    print("None")
    

# 카테고리 0-9 설정해서 백엔드에 드리면 가능한가?

['쿠킹', '베이킹']
